In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [15]:
data_energy = pd.read_csv('../data/consumo_adinelsa_202406.csv', sep=';', encoding="latin1")

In [16]:
data_energy.head()

,ï»¿FECHA_CORTE,PERIODO_FACTURACION,UUID,UBIGEO,DEPARTAMENTO,PROVINCIA,DISTRITO,LOCALIDAD,TARIFA,FECHA_INICIO,CONSUMO_KW,POTENCIA_CONTRATADA,MONTO_SOLES,FASE,CODIGO_CIU
0,20240710,202406,C201812151019901216190122,50701,AYACUCHO,PARINACOCHAS,CORACORA,ASNACOCHA,BT5B,20181215,3.0,3.0,17.2,MONOFASICO,T9700
1,20240710,202406,C201705292074020100180828,50701,AYACUCHO,PARINACOCHAS,CORACORA,CCASACCAHUA,BT5B,20170529,0.0,1.0,49.4,MONOFASICO,T9700
2,20240710,202406,C201802272074070050180828,50708,AYACUCHO,PARINACOCHAS,UPAHUACHO,UPAHUACHO,BT5B,20180227,23.0,1.0,15.2,MONOFASICO,T9700
3,20240710,202406,C201706062074140020180828,50704,AYACUCHO,PARINACOCHAS,PACAPAUSA,ISNO,BT5B,20170606,7.0,1.0,13.3,MONOFASICO,T9700
4,20240710,202406,C201706012074210040180828,50706,AYACUCHO,PARINACOCHAS,PUYUSCA,CCECHCCA,BT5B,20170601,3.0,1.0,7.6,MONOFASICO,T9700


In [17]:
data_energy.columns

Index(['ï»¿FECHA_CORTE', 'PERIODO_FACTURACION', 'UUID', 'UBIGEO',
       'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'LOCALIDAD', 'TARIFA',
       'FECHA_INICIO', 'CONSUMO_KW', 'POTENCIA_CONTRATADA', 'MONTO_SOLES',
       'FASE', 'CODIGO_CIU'],
      dtype='object')

In [18]:
data_energy.UUID.count()

np.int64(80352)

1. Predicción de la producción agrícola (PRODUCCION)
Tipo: Regresión
Objetivo: Predecir la cantidad de producción esperada en una región, dado un cultivo, periodo, superficie, etc.
Útil para:

Planificación agrícola regional

Estimación de abastecimiento

Optimización de recursos

📊 Features útiles:

PERIODO_AGRICOLA, CULTIVO, SUPERFICIE_VERDE, SIEMBRA, COSECHA, RENDIMIENTO, SUPERFICIE_PERDIDA, UBIGEO


class, la sona mas productiva


Perfecto. Vamos a construir una **métrica compuesta** para crear la variable objetivo `EsBuenaOpcion` a partir de tus variables. Esto te permitirá transformar el problema en uno de clasificación binaria sin necesidad de etiquetas externas.

---

## ✅ Paso 1: Variables disponibles

Del dataset:

1. `ubicación_geográfica`
2. `año_inicio_operaciones`
3. `tipo_mercado`
4. `total_puestos`
5. `puestos_funcionamiento`
6. `cantidad_socios`
7. `acceso_alumbrado_publico`
8. `acceso_agua_potable`
9. `acceso_alcantarillado`
10. `acceso_contenedores_residuos`
11. `nro_trabajadores`
12. `ingresos_economicos`

---

## ✅ Paso 2: Métrica compuesta

Creamos un **índice de viabilidad del puesto (IVP)** basado en una puntuación ponderada:

```python
IVP = (
    0.25 * ratio_ocupacion +
    0.20 * acceso_servicios +
    0.15 * estabilidad +
    0.20 * ingresos_norm +
    0.10 * fuerza_laboral +
    0.10 * numero_socios_norm
)
```

### ¿Cómo se calcula cada componente?

| Componente           | Fórmula                                                                      | Tipo          |
| -------------------- | ---------------------------------------------------------------------------- | ------------- |
| `ratio_ocupacion`    | `puestos_funcionamiento / total_puestos` (clamp 0–1)                         | Ratio         |
| `acceso_servicios`   | promedio de \[agua, alcantarillado, alumbrado, residuos] (1 si hay, 0 si no) | Binario (0–1) |
| `estabilidad`        | `año_actual - año_inicio_operaciones` normalizado (ej. /100)                 | Continua      |
| `ingresos_norm`      | `ingresos_economicos` normalizado entre 0 y 1 (min-max o z-score)            | Continua      |
| `fuerza_laboral`     | trabajadores normalizados entre 0 y 1                                        | Continua      |
| `numero_socios_norm` | cantidad\_socios normalizada (0–1)                                           | Continua      |

---

## ✅ Paso 3: Definir la etiqueta `EsBuenaOpcion`

Una vez calculado el `IVP`, definimos la clase:

```python
EsBuenaOpcion = 1 if IVP >= 0.6 else 0
```

(El umbral de 0.6 puede ajustarse con validación cruzada o según tu criterio).

---

## ✅ Paso 4: Manejo del desbalance

En la práctica, puede que haya **pocos puestos realmente buenos**, así que:

* Usa **estrategias para datos desbalanceados**: SMOTE, class weights
* Métricas: usa **F1-score**, **ROC-AUC**, y **costo por tipo de error** si quieres priorizar falsos positivos

---

¿Quieres que te dé el código en Python para implementar esta métrica y generar el target?
